# Lab 1

Group Members:

- Parker

- Suma 

- Chris

- Oliver

## Introduction

**Why is it important to find this kind of bias in machine learning models?**

TODO: Answer this

**Why will the type of investigation I am performing be relevant to other researchers or practitioners?**

TODO: Answer this

For example, we are trying to answer if a romantic comedy is ranked as more positive or if a horror movie is ranked as more negative. 

To do this, we utilize the dataset of IMDB reviews that are pre-labeled as positive or negative and contain movie genres.
We will analyze the bias using the word embeddings of GloVe & ConceptNet. Each of these embeddings will be applied to the movie reviews to determine the overall sentiment of the review. 

We believe that a "lesser" embedding will perform more poorly in the face of a conflicting sentiment lexicon. By this, we expect that "horror" movies may have more "negative" ratings because of the "negative" words used in the reviews to describe the content of the movie, versus the overall context of the review as positive or negative. If the embedding is more narrow-focused, like Glove, it may produce results that bias towards romantic comedies as more positive, since the overall content of the review should have more "positive" words based on the content of the movie. However, we expect an embedding that has a wider knowledge graph focus, like ConceptNet, to remove this bias and focus solely on the review content.

### Loading Embeddings and Lexicons

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import seaborn
import re

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
#this code is taken from LectureNotesMaster/01 ConceptNet.ipynb notebook shared by Prof E Larson/Robyn Speer

def load_embeddings(filename):
    """
    Load a DataFrame from the generalized text format used by word2vec, GloVe,
    fastText, and ConceptNet Numberbatch. The main point where they differ is
    whether there is an initial line with the dimensions of the matrix.
    """
    labels = []
    rows = []
    with open(filename, encoding='utf-8') as infile:
        for i, line in enumerate(infile):
            items = line.rstrip().split(' ')
            if len(items) == 2:
                # This is a header row giving the shape of the matrix
                continue
            labels.append(items[0])
            values = np.array([float(x) for x in items[1:]], 'f')
            rows.append(values)
    
    arr = np.vstack(rows)
    return pd.DataFrame(arr, index=labels, dtype='f')

embeddings = load_embeddings('data/glove.840B.300d.txt')
embeddings.shape

In [ ]:
#Loading the lexicon
def load_lexicon(filename):
    """
    Load a file from Bing Liu's sentiment lexicon
    (https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html), containing
    English words in Latin-1 encoding.
    
    One file contains a list of positive words, and the other contains
    a list of negative words. The files contain comment lines starting
    with ';' and blank lines, which should be skipped.
    """
    lexicon = []
    with open(filename, encoding='latin-1') as infile:
        for line in infile:
            line = line.rstrip()
            if line and not line.startswith(';'):
                lexicon.append(line)
    return lexicon

pos_words = load_lexicon('data/positive-words.txt')
neg_words = load_lexicon('data/negative-words.txt')

print(len(pos_words), len(neg_words))

### Preparing Sentiment Classifier

In [ ]:
#train the model
pos_words_common = list(set(pos_words) & set(embeddings.index)) 
neg_words_common = list(set(neg_words) & set(embeddings.index)) 

pos_vectors = embeddings.loc[pos_words_common]
neg_vectors = embeddings.loc[neg_words_common]
print(pos_vectors.shape,neg_vectors.shape)

In [ ]:
#train the inputs and outputs
vectors = pd.concat([pos_vectors, neg_vectors])
targets = np.array([1 for entry in pos_vectors.index] + [-1 for entry in neg_vectors.index])
labels = list(pos_vectors.index) + list(neg_vectors.index)

In [ ]:
#prep for train test split
train_vectors, test_vectors, train_targets, test_targets, train_labels, test_labels = \
    train_test_split(vectors, targets, labels, test_size=0.1, random_state=0)

In [ ]:
# create a linear classifier 
model = SGDClassifier(loss='log_loss', random_state=0, max_iter=100)
model.fit(train_vectors, train_targets)
accuracy_score(model.predict(test_vectors), test_targets)



### Sentiment Analysis Functions

In [ ]:
def vecs_to_sentiment(vecs):
    # predict_log_proba gives the log probability for each class
    predictions = model.predict_log_proba(vecs)

    # To see an overall positive vs. negative classification in one number,
    # we take the log probability of positive sentiment minus the log
    # probability of negative sentiment.
    # this is a logarithm of the max margin for the classifier, 
    # similar to odds ratio (but not exact) log(p_1/p_0) = log(p_1)-log(p_0)
    return predictions[:, 1] - predictions[:, 0]


def words_to_sentiment(words):
    vecs = embeddings.loc[words].dropna()
    log_odds = vecs_to_sentiment(vecs)
    return pd.DataFrame({'sentiment': log_odds}, index=vecs.index)


# Show 20 examples from the test set
words_to_sentiment(test_labels).iloc[:20]

In [ ]:
#tokenize using regular expressions

TOKEN_RE = re.compile(r"\w.*?\b")
# The regex above finds tokens that start with a word-like character (\w), and continues
# matching characters (.+?) until the next word break (\b). It's a relatively simple
# expression that manages to extract something very much like words from text.


def text_to_sentiment(text):
    # tokenize the input phrase
    tokens = [token.casefold() for token in TOKEN_RE.findall(text)]
    # send each token separately into the embedding, then the classifier
    sentiments = words_to_sentiment(tokens)
    return sentiments['sentiment'].mean() # return the mean for the classifier

### Loading Movie Data

In [ ]:
#Load the movie list for Comedy
df_cat1_movies = pd.read_csv('data/movie_dataset/1_movies_per_genre/Comedy.csv', sep = ',')
df_cat1_movies.head()

In [ ]:
df_cat2_movies = pd.read_csv('data/movie_dataset/1_movies_per_genre/Horror.csv', sep = ',')
df_cat2_movies.head()

In [ ]:
#I feel name, year and genres cols in the csv files are sufficient to retain, drop the rest, add cols back in as needed
df_cat1_movies_lim=df_cat1_movies[['name', 'year',  'genres']]
df_cat2_movies_lim=df_cat2_movies[['name', 'year', 'genres']]

Interesting  problem - there is movie titled Evil Dead thats a Comedy;Horror mix. May be we need load Cat1 to some other genre?

For the movies in the dataframes we have loaded, bring in the reviews

In [ ]:
df_concat_movies = pd.concat([df_cat1_movies_lim, df_cat2_movies_lim])
df_concat_movies['file_name'] = df_concat_movies['name'] + ' ' + df_concat_movies['year'].astype(str) + '.csv'

df_concat_movies.head()

### Combining Movie Reviews

In [ ]:
#https://www.geeksforgeeks.org/how-to-iterate-over-files-in-directory-using-python/
# and https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
import os
directory = 'data/movie_dataset/2_reviews_per_movie_raw'

dfs = list()

# iterate over files in
# that directory
for filename in os.listdir(directory):	
	print('looking for :', filename) 
	if df_concat_movies['file_name'].eq(filename).any():
		print("Found:", df_concat_movies['file_name'])
		data = pd.read_csv(os.path.join(directory, filename), header=None)
		data['file_name'] = filename
#		data['genre'] = df_concat_movies['genres']        
		dfs.append(data)
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df = pd.DataFrame(df)
df = df.rename(columns={"filename": "file_name"})
df_all = df_concat_movies.merge(df)
df_all.columns = df_all.iloc[0]
df_all = df_all[1:]
df_all = df_all.reset_index(drop=True)
# Fixing a weird result from the merge - this seems to work ok
df_all = df_all.rename(columns={"Guardians of the Galaxy": "film name", 2014: "year", 'Comedy; ': "genre", "Guardians of the Galaxy 2014.csv": "file_name"})
df_all = df_all.drop(columns=['file_name','username', 'helpful', 'total', 'date','title'])
df_all['genre'] = df_all['genre'].str.replace(';','',regex=True)
df_all.head()

In [ ]:
df_all.to_csv('df_all.csv', index=False)

In [ ]:
df_all.head()

In [ ]:
# Filter for 'Comedy' genre only (excluding rows that also contain 'Horror')
df_romcom = df_all[df_all['genre'].str.contains('Comedy') & ~df_all['genre'].str.contains('Horror')]

# Filter for 'Horror' genre only (excluding rows that also contain 'Comedy')
df_horror = df_all[df_all['genre'].str.contains('Horror') & ~df_all['genre'].str.contains('Comedy')]

In [ ]:
df_romcom.head()

In [ ]:
df_horror.head()

We have 174941 rows of movie reviews, with 5 columns of information: 
- Film Name
- Year
- Genre
- Rating
- Review


In [ ]:
#TODO: We need to add the header in to the appended reviews file.

## Investigation

As part of your assignment, you will choose a methodology that involves comparing two (or more) techniques to one another. Discuss how you will measure a difference between the two techniques. That is, if you are measuring the difference statistically, what test will you use and why is it appropriate? Are there any limitations to performing this test that you should be aware of?

Source: https://ieee-dataport.org/open-access/imdb-movie-reviews-dataset

Dataset has 1 million reviews from 1150 movies spread across 17 genres; there's also other meta data such as the IMDb rating and movie rating. 

Reference Paper: https://ieeexplore-ieee-org.proxy.libraries.smu.edu/document/9276893 

### Embeddings

### Evaluation metric

F-score

TODO: Explain why

### Hypothesis

TODO: Write hypothesis

## Observations

Carryout your analysis and model training. Explain your steps in as much detail so that the instructor can understand your code. 

Present results from your analysis and provide evidence from the results that support or refute your hypothesis. Write a conclusion based upon the various analyses you performed. Be sure to reference your research questions systematically in your conclusion. With your analysis complete, are there any additional research questions or limitations to your conclusions?

### Loading the dataset

### Cleaning/preparing the dataset

### Model training


### Output

## Conclusion

TODO: Add conclusion

Identify two conferences or journals that would be interested in the results of your analysis.  

## References